# Azure AI Search Knowledge Bases with Blob Knowledge Source

In Azure AI Search, a **knowledge base** is a top-level resource that connects a chat completion model to one or more **knowledge sources** (searchable indexes) for use in agentic retrieval workloads. It defines which data sources to query, which model to use for reasoning, and how query execution should be optimized, powering the `retrieve` method in an LLM-driven information retrieval pipeline. Once created, a knowledge base can be updated at any time, and changes will take effect the next time it’s used. 

This notebook demonstrates how to build **Knowledge Bases** with **knowledge sources**. You’ll learn how agentic retrieval works, how to create and configure a knowledge base, and how to query it for grounded, citation-backed answers while exploring the reasoning behind each step.

## What You'll Learn

In this lab, you will:
* Connect to an existing blob knowledge source that contains indexed documents
* Create a knowledge base that uses Azure OpenAI models for intelligent retrieval
* Query the knowledge base with natural language to get grounded, citation-backed answers
* Explore the agentic knowledge base's activity and reasoning process

## Architecture Overview

The lab environment has already provisioned:
* **Azure Blob Storage** - Contains your source documents (PDFs about employee benefits)
* **Azure AI Search** - Hosts the knowledge source with vectorized and indexed content
* **Azure OpenAI** - Provides embedding and chat completion models

The blob knowledge source uses **integrated vectorization** , a feature that automatically generates vector embeddings from your documents, builds a semantic search index, and connects the content to your knowledge base. This means you don’t need a separate preprocessing step before your data becomes searchable.

## Prerequisites

The following **resources have already been prepared for you**:
- ✅ Azure Blob Storage with uploaded documents
- ✅ Azure AI Search service with knowledge source configured
- ✅ Azure OpenAI with deployed models
- ✅ Indexed documents ready for querying

Let's get started!

## Step 1: Load Environment Variables

> **‼️ Important:** Make sure to open the project on VSCode from  `ignite25-LAB511-build-knowledge-agents...` folder. Opening it from `LAB511` will cause dependency errors.

Before we begin, we need to load the configuration for your Azure resources. The lab setup has created a `.env` file at the workspace root containing:

* Azure AI Search endpoint and credentials  
* Azure OpenAI endpoint, API key, and model deployment names  
* Azure Blob Storage connection information  
* Knowledge source and knowledge base names

Run the cell below to load these environment variables into your Python session.

> **📌 Tip** 
> - The first time you run the cell below, you'll be prompted to select Kernel, select **Install/Enable suggested extensions**, then select **Python Environments** and finally choose the **.venv(3.11.9)** environment that is created for you.
> - You will also be prompted with "Do you want to allow public and private networks to access this app?" Select **Allow**.

In [ ]:
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE", "blob-knowledge-source")
knowledge_agent_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_AGENT", "blob-knowledge-agent")
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
# search blob datasource connection string is optional - defaults to blob connection string
# This field is only necessary if you are using MI to connect to the data source
# https://learn.microsoft.com/azure/search/search-howto-indexing-azure-blob-storage#supported-credentials-and-connection-strings
search_blob_connection_string = os.getenv("SEARCH_BLOB_DATASOURCE_CONNECTION_STRING", blob_connection_string)
blob_container_name = os.getenv("BLOB_CONTAINER_NAME", "documents")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-5")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-5")
use_verbalization = os.getenv("USE_VERBALIZATION", "false") == "true"

# Add a message to indicate that the environment variables have been loaded
print("Environment variables loaded. You can now create the knowledge base.")


## Step 2: Create a blob knowledge source on Azure AI Search

A **Knowledge Source** in Azure AI Search is a searchable index enriched with metadata and vectorized content that the agentic knowledge base can use to retrieve information.Creating a blob knowledge source sets up all necessary resources to index the uploaded document.

Run the cell below to create a blob knowledge source in your Azure AI Search service. This process may take a few minutes.

In [ ]:
from azure.search.documents.indexes.models import (
    AzureSearchIndexKnowledgeSource,
    KnowledgeAgentAzureOpenAIModel,
    AzureOpenAIVectorizerParameters
)
from azure.search.documents.indexes.aio import SearchIndexClient

chat_model = KnowledgeAgentAzureOpenAIModel(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=azure_openai_endpoint,
        deployment_name=azure_openai_chatgpt_deployment,
        api_key=azure_openai_key,
        model_name=azure_openai_chatgpt_model_name
    )
)

# Create knowledge source for hrdocs index
hrdocs_knowledge_source = AzureSearchIndexKnowledgeSource(
    name="hrdocs-knowledge-source",
    index_name="hrdocs"
)

# Create knowledge source for healthdocs index
healthdocs_knowledge_source = AzureSearchIndexKnowledgeSource(
    name="healthdocs-knowledge-source",
    index_name="healthdocs"
)

async with SearchIndexClient(endpoint=endpoint, credential=credential) as client:
    await client.create_or_update_knowledge_source(hrdocs_knowledge_source)
    print(f"Created knowledge source: {hrdocs_knowledge_source.name}")
    
    await client.create_or_update_knowledge_source(healthdocs_knowledge_source)
    print(f"Created knowledge source: {healthdocs_knowledge_source.name}")

## Step 3: Create a Knowledge Base

A **Knowledge Base** is an intelligent layer that connects your indexed data to a language model. Instead of just returning search results, it plans queries, retrieves relevant data, and generates grounded answers.

In this step, we'll create a knowledge base that acts as an intelligent wrapper around your knowledge source and LLM deployment.

### Output Modality Options

The knowledge base supports two output modalities:
* **`EXTRACTIVE_DATA`** (default) - Returns exact content from your knowledge sources without generative alteration
* **`ANSWER_SYNTHESIS`** (used here) - Generates natural language answers using the LLM that cite the retrieved content

We're using `ANSWER_SYNTHESIS` with `include_activity=True` to get:
* **LLM-generated answers** that cite retrieved documents
* **Detailed activity logs** showing the knowledge base's reasoning process, including subqueries and re-ranking decisions

Learn more about [answer synthesis in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-synthesize).

Run the cell below to create the knowledge base.

In [ ]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeSourceReference, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality

output_config = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True
)

agent = KnowledgeAgent(
    name=knowledge_agent_name,
    models=[chat_model],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=hrdocs_knowledge_source.name,
            include_reference_source_data=True,
            always_query_source=True
        ),
        KnowledgeSourceReference(
            name=healthdocs_knowledge_source.name,
            include_reference_source_data=True,
            always_query_source=True
        )
    ],
    output_configuration=output_config
)

async with SearchIndexClient(endpoint=endpoint, credential=credential) as index_client:
    await index_client.create_or_update_agent(agent)
    print(f"Created knowledge agent: {agent.name}")

## Step 4: Query the Knowledge Base

Now it’s time to query our documents and see agentic retrieval in action. This step demonstrates how the knowledge base processes queries to produce grounded, citation-backed answers.

### How Agentic Retrieval Works

Given a user query and conversation history, the knowledge base:
1. **Analyzes the conversation** - Understands the full context and user's information need
2. **Query decomposition** - Breaks down complex queries into focused subqueries
3. **Concurrent execution** - Runs subqueries in parallel against your knowledge source
4. **Semantic reranking** - Uses semantic ranker to rerank and filter results for relevance
5. **Answer synthesis** - Synthesizes the top results into a natural-language answer with citations

### Example Query

The example below asks about differences between two employee benefit plans. The agentic knowledge base will:
* Search across the indexed PDF documents
* Find relevant sections from both plans
* Generate a comparative answer with specific citations

Try running the cell below, then modify the `text` field in the `messages` list to ask your own questions!

In [ ]:
from azure.search.documents.agent.aio import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent
import warnings

# Suppress aiohttp warnings
warnings.filterwarnings('ignore', category=ResourceWarning)

messages = [
    KnowledgeAgentMessage(
        role="user",
        content=[KnowledgeAgentMessageTextContent(
            text="Differences between Northwind Health Plus and Standard"
        )]
    )
]

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=knowledge_agent_name, credential=credential)
try:
    print("🔍 Querying the knowledge base...")
    result = await agent_client.retrieve(KnowledgeAgentRetrievalRequest(messages=messages))
    print("✅ Query successful! The agentic knowledge base has retrieved relevant information.")
    print("📊 Run the cells in Step 5 below to review the response, activity, and references.")
finally:
    await agent_client.close()

## Step 5: Review the Retrieval Response

The knowledge base's response contains three key components that provide full transparency into the retrieval process:

### 1. Response Content (Answer)
An LLM-generated answer to your query that cites the retrieved documents. This is the primary output you'd show to end users.

### 2. Activity Content (Reasoning)
Detailed planning and execution information showing:
* **Subqueries generated** - How the agentic knowledge base broke down your query
* **Reranking decisions** - Which results were promoted or filtered
* **Intermediate steps** - The agentic knowledge base's thought process and execution flow

### 3. References Content (Sources)
Source documents and text chunks that contributed to the answer, including:
* Document names and metadata
* Specific text passages used
* Relevance scores and rankings

### Why This Matters

These three components enable you to:
* **Verify grounding** - Ensure answers are based on your actual content
* **Build traceable citations** - Create links back to source documents
* **Debug and optimize** - Understand why certain results were retrieved
* **Tune retrieval parameters** - Adjust reranker thresholds and knowledge source settings

> **💡 Tip:** Retrieval parameters (like reranker thresholds and knowledge source parameters) influence how aggressively your agentic knowledge base reranks results and which sources it queries. Inspect the activity and references to validate grounding quality.

Let's examine each component! 

Run the first cell below to display the **synthesized answer** with citations:

In [ ]:
print("Agentic knowledge base's Response (synthesized answer):")
print(result.response[0].content[0].text)

Run the cell below to display the **agentic knowledge base's activity log** showing query decomposition and reasoning:

In [ ]:
import json

# Activity -> JSON string of activity as list of dicts

activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("activity_content:\n", activity_content, "\n")

Run the cell below to display the **source references** used to generate the answer:

In [ ]:
# References -> JSON string of references as list of dicts
references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
print("references_content:\n", references_content, "\n")

## Notebook Complete! 🎉

Congratulations! You've successfully completed the hands-on portion of this lab.

### What You Learned: The Agentic Knowledge Base Orchestration Layer

This lab focused on the **intelligence layer** that sits on top of search infrastructure. Here's the real depth of what you explored:

#### **1. Knowledge Bases as Orchestrators**
You built a knowledge base that doesn't just search—it **orchestrates** an entire retrieval workflow. The knowledge base coordinates between your knowledge sources (the indexed blob data) and the LLM (GPT model) to produce grounded, citation-backed answers.

#### **2. Answer Synthesis vs. Raw Retrieval**
Traditional search returns chunks of text. You configured **Answer Synthesis mode**, where the LLM reads retrieved passages and generates a coherent natural-language response with inline citations. This is the foundation of modern copilots and chat interfaces over enterprise data.

#### **3. Query Decomposition & Planning**
When you examined the **activity logs**, you saw how the knowledge base breaks complex queries into focused subqueries, executes them in parallel, and applies semantic reranking. This is agentic behavior—the system plans and adapts based on the query context.

#### **4. Grounding & Transparency**
Every answer is backed by source references. You explored how to trace responses back to specific documents and passages, enabling trust and auditability in AI systems. The activity logs show exactly how the knowledge base arrived at its answer.

#### **5. The RAG Evolution**
This lab demonstrated **Retrieval-Augmented Generation (RAG)** at scale. The knowledge base doesn't just retrieve—it reasons about what to retrieve, how to combine results, and how to synthesize coherent answers. This is next-level RAG: agentic, explainable, and production-ready.

### Next Steps

📖 **Return to the lab instructions page** to have a look at the summary and reflect on how **agentic behavior elevates RAG beyond simple search**, explore additional learning resources, and understand how to apply these patterns to real-world enterprise AI solutions.

💡 **Experiment more!** Go back to Step 4 and try different queries—watch how the agentic knowledge base adapts its retrieval strategy based on query complexity.